<a href="https://colab.research.google.com/github/ptran1203/style_transfer/blob/master/Adain_Style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
from google.colab import drive, output
data_loaded = False
drive.mount('/content/drive')
BASE_DIR = "/content/drive/My Drive/Style_Transfer"
!rm -rf '/content/style_transfer'
!git clone https://github.com/ptran1203/style_transfer

TensorFlow 1.x selected.
Mounted at /content/drive
Cloning into 'style_transfer'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 352 (delta 18), reused 77 (delta 12), pack-reused 269
Receiving objects: 100% (352/352), 56.37 MiB | 10.98 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [2]:
cd style_transfer

/content/style_transfer


In [3]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pickle
import os
import utils
from dataloader import DataGenerator
from model import *

class DataGen(DataGenerator):
    BATCH_FILES= 4

class SModel(StyleTransferModel):
    def compute_style_loss(self, gen_img, style_img):
        gen_feats = self.style_layers(gen_img)
        style_feats = self.style_layers(style_img)
        style_loss = []
        axis = [1, 2]
        for i in range(len(style_feats)):
            gmean = K.mean(gen_feats[i], axis=axis)
            gstd = K.std(gen_feats[i], axis=axis)

            smean = K.mean(style_feats[i], axis=axis)
            sstd = K.std(style_feats[i], axis=axis)

            style_loss.append(
                K.sum(K.square(gmean - smean)) +
                K.sum(K.square(gstd - sstd))
            )

        return Reduction()(style_loss)

    def build_decoder(self, input_shape):
        feat = Input(input_shape)
        kernel_size = 3

        x = self.conv_block(feat, 512, kernel_size=kernel_size, up_sampling=True)

        x = self.conv_block(x, 256, kernel_size=kernel_size)
        x = self.conv_block(x, 256, kernel_size=kernel_size)
        x = self.conv_block(x, 256, kernel_size=kernel_size)
        x = self.conv_block(x, 256, kernel_size=kernel_size, up_sampling=True)

        # x = self.conv_block(x, 128, kernel_size=kernel_size)
        # x = self.conv_block(x, 128, kernel_size=kernel_size)
        x = self.conv_block(x, 128, kernel_size=kernel_size)
        x = self.conv_block(x, 128, kernel_size=kernel_size, up_sampling=True)

        x = self.conv_block(x, 64, kernel_size=kernel_size)
        x = self.conv_block(x, 64, kernel_size=kernel_size)

        style_image = self.conv_block(x, 3, kernel_size=kernel_size, activation='linear')

        model = Model(inputs=feat, outputs=style_image, name='decoder')
        return model




style_layer_names=[
    'block1_conv1', 'block2_conv1',
    'block3_conv1', 'block4_conv1',
]
last_layer='block4_conv1'
pre_trained_model = 'vgg16'
rst = 256
data_gen = DataGen(BASE_DIR, 8, rst=rst, max_size=1500, multi_batch=False,
                   normalize=True)
smodel = SModel(BASE_DIR, None, 1e-4,
                style_layer_names=style_layer_names,
                last_layer=last_layer, 
                show_interval=5,
                style_loss_weight=3.5,
                pre_trained_model=pre_trained_model)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

58892288/58889256 [==============================] - 6s 0us/step
Encoder: vgg16


In [ ]:
smodel.load_weight()
smodel.train(data_gen, 100, augment_factor=0)
smodel.save_weight()

Train on 1500 samples
Train epochs 1/100 - WARNING:tensorflow:From /tensorflow-1.15.2/python3.6/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [ ]:
smodel.load_weight()

In [ ]:
# smodel.save_weight()
idx = np.random.randint(0, 300)
sid = 15
# cimg = data_gen.x[idx:idx+1]
cimg = utils.http_get_img('https://github.com/elleryqueenhomels/arbitrary_style_transfer/blob/master/images/content/stata.jpg?raw=true', None)
simg =  data_gen.y[sid:sid+1]
gen = smodel.generate(cimg, simg)[0]
gen = utils.deprocess(gen)
cv2_imshow(gen)

In [ ]:
urls = [
    # 'https://github.com/elleryqueenhomels/arbitrary_style_transfer/raw/master/images/style_thumb/escher_sphere_thumb.jpg',
    'https://1.bp.blogspot.com/-39XZDC2-ifQ/UxXqyjWlt-I/AAAAAAAAXzc/_rTksQDKmx8/s1600/Gleizes,+Albert,+Pablo_Picasso,+1909-10,+Figure++in+a+Chair,+Seated+Nude,+Woman.jpg',
    'https://github.com/elleryqueenhomels/arbitrary_style_transfer/raw/master/images/style_thumb/udnie_thumb.jpg',
    'https://github.com/elleryqueenhomels/arbitrary_style_transfer/raw/master/images/style_thumb/mosaic_thumb.jpg',
    'https://github.com/elleryqueenhomels/arbitrary_style_transfer/raw/master/images/style_thumb/cat_thumb.jpg'
]
cv2_imshow(utils.deprocess(utils.de_norm(cimg[0])))
for url in urls:
    simg = utils.http_get_img(url, None)
    cv2_imshow(utils.deprocess(utils.de_norm(simg[0])))
    cv2_imshow(utils.deprocess(utils.de_norm(smodel.generate(cimg, simg)[0])))

In [ ]:
from classification_models.keras import Classifiers

vggnet, _ = Classifiers.get('vgg19')
input_shape = (None, None, 3)
model = vggnet(input_shape=input_shape,
            weights='imagenet',
            include_top=False)

model.summary()

In [ ]:
from keras.applications.vgg19 import preprocess_input

cv2_imshow((data_gen.y[0]))
cv2_imshow(utils.deprocess(utils.de_norm(data_gen.y[0])))

